In [4]:
import requests, datetime, time, h5py, json
import geopy.distance
from uszipcode import Zipcode, SearchEngine

import re, numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression, LassoCV, RidgeCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB

from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import stop_words
import nltk
from nltk.tokenize import RegexpTokenizer 
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag, word_tokenize

%matplotlib inline
pd.options.display.max_columns = 90
pd.options.display.max_rows = 200

In [1]:
features = ['alias', 'price', 'categories', 'coordinates', 'review_count',
            'display_address', 'rating', 'distance', 'url']
features_added = ['postal_code','main_category','latitude','longitude','city']

search_radius = 1000  # in meters, 1600m is about 1 mile
review_count_cutoff = 10

main_categories = ['Active Life','Arts & Entertainment','Automotive','Beauty & Spas',
'Education','Event Planning & Services','Financial Services','Food','Health & Medical',
'Home Services','Hotels & Travel','Local Services','Mass Media','Nightlife','Pets',
'Professional Services','Public Services & Government','Real estate','Religious Organizations',
'Restaurants','Shopping']

main_alias = ['active','arts','Automotive','beautysvc',
'Education','eventservices','financialservices',
'Food','health','homeservices','hotelstravel','localservices',
'massmedia','Nightlife','Pets','professional','publicservicesgovt',
'realestate','religiousorgs','Restaurants','Shopping']

In [ ]:
def getPriceRange(string) :
    regex = re.compile("RestaurantsPriceRange2\\': \\'(\d)")
    try :
        match = regex.search(string)
        return int(match.groups(0)[0])
    except : return 0

In [2]:
def calcDist(lat1, long1, lat2, long2) :  # returns distance in meters
    try : d = geopy.distance.distance((lat1,long1), (lat2,long2)).m
    except : d = np.nan
    return d

In [ ]:
def calcDistances(lat, long, citydf) :
    distances = pd.Series(index=citydf.index)
    for index in citydf.index :
        distances[index] = calcDist(lat, long, citydf.loc[index,'latitude'], citydf.loc[index,'longitude'])
    return distances

In [ ]:
def latlong_tozipcode(tupl) :
    try :
        (lat, long) = tupl
        result = search.by_coordinates(lat, long, radius=10, returns=1)
        int(result[0].to_dict()['zipcode'])
    except : return 0
    return int(result[0].to_dict()['zipcode'])